In [1]:
import cv2 as cv
import mediapipe as mp
import random #컴퓨터의 수를 랜덤으로 가져오기 위해서 

#User
def user_choice(hand_landmarks):
    landmarks = hand_landmarks.landmark
    thumb_tip = landmarks[mp_hand.HandLandmark.THUMB_TIP]
    index_finger_tip = landmarks[mp_hand.HandLandmark.INDEX_FINGER_TIP]
    middle_finger_tip = landmarks[mp_hand.HandLandmark.MIDDLE_FINGER_TIP]
    ring_finger_tip = landmarks[mp_hand.HandLandmark.RING_FINGER_TIP]
    pinky_tip = landmarks[mp_hand.HandLandmark.PINKY_TIP]

    thumb_mcp = landmarks[mp_hand.HandLandmark.THUMB_MCP]
    index_finger_mcp = landmarks[mp_hand.HandLandmark.INDEX_FINGER_MCP]
    middle_finger_mcp = landmarks[mp_hand.HandLandmark.MIDDLE_FINGER_MCP]
    ring_finger_mcp = landmarks[mp_hand.HandLandmark.RING_FINGER_MCP]
    pinky_mcp = landmarks[mp_hand.HandLandmark.PINKY_MCP]

    #보
    if thumb_tip.y < thumb_mcp.y and index_finger_tip.y < index_finger_mcp.y and \
    middle_finger_tip.y < middle_finger_mcp.y and ring_finger_tip.y < ring_finger_mcp.y and pinky_tip.y < pinky_mcp.y:
        return "Paper"
    #바위
    elif thumb_tip.y < index_finger_tip.y and thumb_tip.y < middle_finger_tip.y and \
    thumb_tip.y < ring_finger_tip.y and thumb_tip.y < pinky_tip.y:  
        return "Rock"
    #가위
    elif index_finger_tip.y < thumb_tip.y and index_finger_tip.y < ring_finger_tip.y and \
    index_finger_tip.y < pinky_tip.y and middle_finger_tip.y < thumb_tip.y and \
    middle_finger_tip.y < ring_finger_tip.y and middle_finger_tip.y < pinky_tip.y:
        return "Scissors"
    
    else:  
        return "null"

#Computer
def computer_choice():
    return random.choice(["Rock", "Paper", "Scissors"])

#추가한 부분
#winner
def winner(user_choice, computer_choice):
    if user_choice=="null":
        return "Do it again"
    
    elif user_choice == computer_choice:
        return "Draw"
    
    elif (user_choice == "Rock" and computer_choice == "Scissors") or \
         (user_choice == "Scissors" and computer_choice == "Paper") or \
         (user_choice == "Paper" and computer_choice == "Rock"):
        return "User wins"
    else:
        return "Computer wins"

mp_hand = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

hand = mp_hand.Hands(max_num_hands=2, static_image_mode=False, min_detection_confidence=0.5, min_tracking_confidence=0.5)
cap = cv.VideoCapture(0, cv.CAP_DSHOW)

while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to capture frame. Exiting.")
        break

    frame_rgb = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    res = hand.process(frame_rgb)

    if res.multi_hand_landmarks:
        for landmarks in res.multi_hand_landmarks:
            mp_drawing.draw_landmarks(frame, landmarks, mp_hand.HAND_CONNECTIONS, 
                                      mp_styles.get_default_hand_landmarks_style(), 
                                      mp_styles.get_default_hand_connections_style())
            
            
            user_hand_shape = user_choice(landmarks)
            cv.putText(frame, f"User: {user_hand_shape}", (10, 50), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv.LINE_AA)
            
        
            computer_hand_shape = computer_choice()
            cv.putText(frame, f"Computer: {computer_hand_shape}", (10, 100), cv.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv.LINE_AA)
            
            result =winner(user_hand_shape, computer_hand_shape)
            cv.putText(frame, result, (10, 150), cv.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv.LINE_AA)

                
    cv.imshow("MediaPipe Hands", frame)
    if cv.waitKey(5) == ord('q'):
        break

cap.release()
cv.destroyAllWindows()
